## Real experiments for head movements

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

In [51]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score,f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.dummy import DummyClassifier
from seqlearn.perceptron import StructuredPerceptron
import sklearn_crfsuite
from sklearn_crfsuite import metrics as skcrfmetrics
from scipy.stats import pearsonr,spearmanr

In [52]:
import glob

In [53]:
windowsize="9-11-13"
memory="middle"

PATH = "C:\\Users\\zgk261\\nomco\\"+windowsize+"\\"+memory+"\\"
PATH

'C:\\Users\\zgk261\\nomco\\9-11-13\\middle\\'

In [54]:
!dir /B C:\Users\zgk261\nomco\9-11-13\middle\

F2_M4-all-final2_primary_F2.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F2_M4-all-final2_primary_M4.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F3_F6-all-final2_primary_F3.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F3_F6-all-final2_primary_F6.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F4_F1-all-final2_primary_F1.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F4_F1-all-final2_primary_F4.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F5_F2-all-final2_primary_F2.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBac

### SKELETON TO READ FILES, SPEAKERS and PRIMARY SPEAKERS


In [55]:
speaker_set = set()

for each_file in glob.glob(PATH+"*.tab"):
    filename=each_file.split("\\")[-1]
    fileinfo=filename.split("-")
    
    primary_speaker = fileinfo[2].split("_")[-1][:-2] #fileinfo[2] = final2_primary_M1.T
    speakers = fileinfo[0]
    l_speaker,r_speaker = speakers.split("_")

    speaker_set.add(primary_speaker)

In [56]:
speaker_set

{'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6'}

### SKELETON TO READ FILES AND COSS_VALIDATE

In the following way: In each iteration, we will leave one speaker out (two files), train a model and test on those two files.

In [57]:

for speaker_on_focus in speaker_set:
    train_files = []
    test_files  = []
    
    for each_file in glob.glob(PATH+"*.tab"):
        filename=each_file.split("\\")[-1]
        fileinfo=filename.split("-")

        primary_speaker = fileinfo[2].split("_")[-1][:-2] #fileinfo[2] = final2_primary_M1.T
        speakers = fileinfo[0]
        l_speaker,r_speaker = speakers.split("_")
        if primary_speaker == l_speaker:
            secondary_speaker = r_speaker
        else:
            secondary_speaker = l_speaker

        #print (primary_speaker,l_speaker,r_speaker,speaker_on_focus)

        file_df = pd.read_csv(each_file,sep="\t")
        file_df.columns = [el.replace(primary_speaker, "PRIMA").replace(secondary_speaker, "SECON") for el in file_df.columns]

        if primary_speaker != speaker_on_focus:
            #FILES TO TRAIN!
            train_files.append(file_df)
        else:
            #FILES TO TEST!
            test_files.append(file_df)
            
        
        
    training_data = pd.concat(train_files, ignore_index=True)
    testing_data = pd.concat(test_files, ignore_index=True)
    print ("Speaker on focus:",speaker_on_focus)
    print ("Training data size:",training_data.shape)
    print ("Testing data size:",testing_data.shape)
    print ()


Speaker on focus: F5
Training data size: (181647, 28)
Testing data size: (16413, 28)

Speaker on focus: M6
Training data size: (181528, 28)
Testing data size: (16532, 28)

Speaker on focus: M4
Training data size: (181548, 28)
Testing data size: (16512, 28)

Speaker on focus: F3
Training data size: (181651, 28)
Testing data size: (16409, 28)

Speaker on focus: M1
Training data size: (180025, 28)
Testing data size: (18035, 28)

Speaker on focus: M5
Training data size: (180649, 28)
Testing data size: (17411, 28)

Speaker on focus: F4
Training data size: (181137, 28)
Testing data size: (16923, 28)

Speaker on focus: M3
Training data size: (181426, 28)
Testing data size: (16634, 28)

Speaker on focus: F1
Training data size: (181919, 28)
Testing data size: (16141, 28)

Speaker on focus: F6
Training data size: (182822, 28)
Testing data size: (15238, 28)

Speaker on focus: M2
Training data size: (182715, 28)
Testing data size: (15345, 28)

Speaker on focus: F2
Training data size: (181593, 28)


### Experiments:

It seems that in the 3-6-9 experiments considering both the past and future of frames, the MLP was not the best model, but the Logistic Regression. I think, though, that the MLP will show good performance when cross-validating with the whole data.

I will, then, try 3 classifiers:

  * SVM, because it is the default classifier that is commonly used
  * Logistic Regression, because it seems to perform the best in our initial experiments with two dialogue data
  * MLP, because when including more data, I expect to estimate a more general (and thus, better) model

In [58]:
accuracy_results = pd.DataFrame(0,columns=speaker_set,index=["MF"])
accuracy_results

,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0,0,0,0,0,0,0,0,0,0,0,0


In [59]:
import math

def map_movements(mov_class):
    if mov_class == "Nod" or mov_class == "Jerk":
        return 1
    elif mov_class == "Shake":
        return 2
    elif mov_class!=mov_class: #This returns True if the value is NAN, and it works. It might be source of errors.
        return 0
    else:
        return 3



In [60]:
accuracy_results = pd.DataFrame(0,columns=speaker_set,index=["MF"])
f1score_results = pd.DataFrame(0,columns=speaker_set,index=["MF"])
f1score_resultsw = pd.DataFrame(0,columns=speaker_set,index=["MF"])
#f1score_resultsb = pd.DataFrame(0,columns=speaker_set,index=["MF"])
#f1_score(y_true, y_pred, average='micro')

for speaker_on_focus in speaker_set:
    train_files = []
    test_files  = []
    
    for each_file in glob.glob(PATH+"*.tab"):
        filename=each_file.split("\\")[-1]
        fileinfo=filename.split("-")

        primary_speaker = fileinfo[2].split("_")[-1][:-2] #fileinfo[2] = final2_primary_M1.T
        speakers = fileinfo[0]
        l_speaker,r_speaker = speakers.split("_")
        if primary_speaker == l_speaker:
            secondary_speaker = r_speaker
        else:
            secondary_speaker = l_speaker

        #print (primary_speaker,l_speaker,r_speaker,speaker_on_focus)

        file_df = pd.read_csv(each_file,sep="\t")
        file_df.columns = [el.replace(primary_speaker, "PRIMA").replace(secondary_speaker, "SECON") for el in file_df.columns]

        if primary_speaker != speaker_on_focus:
            #FILES TO TRAIN!
            train_files.append(file_df)
        else:
            #FILES TO TEST!
            test_files.append(file_df)
            
        
        
    training_data = pd.concat(train_files, ignore_index=True)
    testing_data = pd.concat(test_files, ignore_index=True)
    print ("Speaker on focus:",speaker_on_focus)
    print ("Training data size:",training_data.shape)
    print ("Testing data size:",testing_data.shape)
    print ()
    
    
    
    
    
    
    
    #Train data
    features=['PRIMA:velocity-r', 'PRIMA:velocity-clock', 'PRIMA:velocity-x', 'PRIMA:velocity-y',
              'PRIMA:acceleration-r', 'PRIMA:acceleration-clock', 'PRIMA:acceleration-x','PRIMA:acceleration-y'
             ]

    feats_learning = training_data[features]
    X = feats_learning.dropna() #REMEMBER THIS!!!! We're ignoring any row that has a NaN value.
    scaler = MinMaxScaler((0,1))
    scaler.fit(X)
    X_norm = scaler.transform(X)
    #X_norm = X
    #We're ignoring anyrow that has a NaN value.
    Y = (training_data[(feats_learning.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].notna()*1).values #0: no movement; 1: movement
    #Y = training_data[(feats_learning.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].map(map_movements)
    
    #Test data
    feats_testing = testing_data[features]

    X_test = feats_testing.dropna() #REMEMBER THIS!!!! We're ignoring any row that has a NaN value.
    X_test_norm = scaler.transform(X_test)
    #X_test_norm = X_test
    Y_test = (testing_data[(feats_testing.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].notna()*1).values #0: no movement; 1: movement
#    Y_test =  testing_data[(feats_testing.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].map(map_movements)
    
    classifiers_to_test = [(DummyClassifier(strategy="most_frequent"),"MF"),
                           (LogisticRegression(solver="liblinear"),"LR"),
                          (LinearSVC(),"LINEARSVC"),
                          (MLPClassifier(hidden_layer_sizes=(30,30,30,30), random_state=442), "MLP")
                          ]
                        #I think that now we have to fine-tune the n_hidden_layers and their sizes (MLP)
    
    
    for clf, clfname in classifiers_to_test:
        print ("Trying classifier ",clfname,speaker_on_focus)
        clf.fit(X_norm, Y)
        y_pred = clf.predict(X_test_norm)
        accuracy_results.loc[clfname,speaker_on_focus] = accuracy_score(Y_test,y_pred)
        #f1score_resultsb.loc[clfname,speaker_on_focus] = f1_score(Y_test,y_pred)
        f1score_results.loc[clfname,speaker_on_focus] = f1_score(Y_test,y_pred, average='macro')
        f1score_resultsw.loc[clfname,speaker_on_focus] = f1_score(Y_test,y_pred, average='weighted')
    
        print (confusion_matrix(Y_test,y_pred))
        print (accuracy_score(Y_test,y_pred)) #same as f1-score (micro-averagaed)
        print (f1_score(Y_test,y_pred, average='macro'))
        print (f1_score(Y_test,y_pred, average='weighted'))
        y_pred=None
        print ()

        
accuracy_results

Speaker on focus: F5
Training data size: (181647, 28)
Testing data size: (16413, 28)

Trying classifier  MF F5


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[12154     0]
 [ 4259     0]]
0.7405105708889295
0.42545594567157907
0.6301092504347008

Trying classifier  LR F5
[[11384   770]
 [ 3257  1002]]
0.7546457076707488
0.5909969851914886
0.7154437838790016

Trying classifier  LINEARSVC F5
[[11498   656]
 [ 3356   903]]
0.7555596173764698
0.5809336861497003
0.7110584362238244

Trying classifier  MLP F5
[[10565  1589]
 [ 2686  1573]]
0.7395357338694937
0.627829061403087
0.7259078179780738

Speaker on focus: M6
Training data size: (181528, 28)
Testing data size: (16532, 28)

Trying classifier  MF M6


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[9316    0]
 [7216    0]]
0.5635131865473022
0.36041473228102755
0.4061969085325493

Trying classifier  LR M6
[[8324  992]
 [5184 2032]]
0.6264214856036777
0.5631413205397827
0.5842615282057362

Trying classifier  LINEARSVC M6
[[8412  904]
 [5377 1839]]
0.6200701669489475
0.5487341336875693
0.5715251986273067

Trying classifier  MLP M6
[[7963 1353]
 [4102 3114]]
0.6700338736994919
0.638974597102672
0.6524257170971702

Speaker on focus: M4
Training data size: (181548, 28)
Testing data size: (16512, 28)

Trying classifier  MF M4


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[10965     0]
 [ 4380     0]]
0.7145650048875856
0.4167616875712657
0.5956066346326397

Trying classifier  LR M2
[[10584   381]
 [ 3446   934]]
0.7506028022157054
0.5874482007902898
0.6987821956489801

Trying classifier  LINEARSVC M2
[[10652   313]
 [ 3539   841]]
0.7489736070381232
0.5754073906614557
0.6919025017449696

Trying classifier  MLP M2
[[9886 1079]
 [2686 1694]]
0.7546432062561095
0.6568432540284872
0.7354580837915357

Speaker on focus: F2
Training data size: (181593, 28)
Testing data size: (16467, 28)

Trying classifier  MF F2


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[10315     0]
 [ 6152     0]]
0.6264043237991134
0.38514674034799495
0.48251516690223695

Trying classifier  LR F2
[[9037 1278]
 [3108 3044]]
0.7336491164146475
0.692984153952177
0.7212318160131256

Trying classifier  LINEARSVC F2
[[9216 1099]
 [3341 2811]]
0.7303698305702314
0.6823060091441457
0.7135456170437234

Trying classifier  MLP F2
[[7686 2629]
 [1816 4336]]
0.7300661929920447
0.7184122076918418
0.7328944586783328



,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0.740511,0.563513,0.648316,0.588701,0.692986,0.551088,0.591385,0.719731,0.671644,0.506497,0.714565,0.626404
LR,0.754646,0.626421,0.691194,0.643123,0.740283,0.637930,0.731076,0.743838,0.757512,0.683686,0.750603,0.733649
LINEARSVC,0.755560,0.620070,0.688045,0.634652,0.737455,0.631153,0.721562,0.742996,0.751440,0.675942,0.748974,0.730370
MLP,0.739536,0.670034,0.714511,0.688951,0.750208,0.690885,0.768658,0.743537,0.780683,0.688542,0.754643,0.730066


In [61]:
accuracy_results

,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0.740511,0.563513,0.648316,0.588701,0.692986,0.551088,0.591385,0.719731,0.671644,0.506497,0.714565,0.626404
LR,0.754646,0.626421,0.691194,0.643123,0.740283,0.637930,0.731076,0.743838,0.757512,0.683686,0.750603,0.733649
LINEARSVC,0.755560,0.620070,0.688045,0.634652,0.737455,0.631153,0.721562,0.742996,0.751440,0.675942,0.748974,0.730370
MLP,0.739536,0.670034,0.714511,0.688951,0.750208,0.690885,0.768658,0.743537,0.780683,0.688542,0.754643,0.730066


In [62]:
f1score_resultsw

,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0.630109,0.406197,0.509992,0.436293,0.567316,0.391594,0.439536,0.602434,0.539715,0.340577,0.595607,0.482515
LR,0.715444,0.584262,0.650303,0.578222,0.727355,0.575815,0.713655,0.717499,0.729791,0.678059,0.698782,0.721232
LINEARSVC,0.711058,0.571525,0.640178,0.560727,0.720856,0.562615,0.699826,0.709976,0.718796,0.667214,0.691903,0.713546
MLP,0.725908,0.652426,0.704671,0.665722,0.744415,0.664098,0.767050,0.742132,0.775335,0.687785,0.735458,0.732894


In [63]:
f1score_results

,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0.425456,0.360415,0.393320,0.370555,0.409328,0.355291,0.371616,0.418514,0.401786,0.336208,0.416762,0.385147
LR,0.590997,0.563141,0.589811,0.539461,0.667122,0.556380,0.694267,0.624439,0.672518,0.677478,0.587448,0.692984
LINEARSVC,0.580934,0.548734,0.575250,0.518852,0.656485,0.541963,0.678251,0.609779,0.657018,0.666484,0.575407,0.682306
MLP,0.627829,0.638975,0.667654,0.643001,0.693563,0.652256,0.757578,0.678706,0.740058,0.687960,0.656843,0.718412


In [64]:
accuracy_results.T.describe()

,MF,LR,LINEARSVC,MLP
count,12.000000,12.000000,12.000000,12.000000
mean,0.634612,0.707830,0.703185,0.726688
std,0.075262,0.049181,0.051141,0.035664
min,0.506497,0.626421,0.620070,0.670034
25%,0.582404,0.673545,0.665619,0.690402
50%,0.637360,0.732363,0.725966,0.734801
75%,0.698381,0.745529,0.744491,0.751317
max,0.740511,0.757512,0.755560,0.780683


In [65]:
f1score_results.T.describe()

,MF,LR,LINEARSVC,MLP
count,12.000000,12.000000,12.000000,12.000000
mean,0.387033,0.621337,0.607622,0.680236
std,0.028472,0.056970,0.058247,0.041100
min,0.336208,0.539461,0.518852,0.627829
25%,0.368020,0.581371,0.568621,0.649942
50%,0.389234,0.607718,0.595356,0.673180
75%,0.411186,0.673758,0.659385,0.699775
max,0.425456,0.694267,0.682306,0.757578
